## Physical Review B , 106, 035432 (2022)
Reproducing results presented in https://journals.aps.org/prb/pdf/10.1103/PhysRevB.106.035432

In [4]:
%load_ext autoreload

import kwant
import numpy as np
import matplotlib.pyplot as plt 
import kwant.continuum
import scipy.sparse.linalg as sla
import tinyarray as tiny
import matplotlib as mpl
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

%matplotlib widget
mpl.rcParams.update({'font.size': 20, 'text.usetex': True,  'font.family': 'serif', 'font.serif': ['Computer Modern']})

sx = tiny.array([[0 , 1] , [1 , 0]])
sy = tiny.array([[0 , -1j] , [1j , 0]])
sz = tiny.array([[1 , 0] , [0 , -1]])


I = tiny.array([[1 , 0] , [0 , 1]])
t_plus = 0.5*(np.kron(sx , I) + 1j*np.kron(sy , I))
t_minus = t_plus.T
phs = np.kron(sy , sy)

/Users/krishnanganesh/anaconda3/lib/python3.8/site-packages/kwant/solvers/default.py:16: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  warnings.warn("MUMPS is not available, "


In [27]:
from dataclasses import dataclass
@dataclass
class system_parameters:
    t : float # Hopping integral
    W: float # Width of normal region
    L: float # Length of normal region
    a : float # Lattice constant
    mu : float # Chemical potential
    Delta: float # SUperconducting gap.
    flux: float # Flux throught the normal region.
    phase: float  # Josephson phase difference. 

params = system_parameters(t =1 , W = 20 , L = 5 , a = 1 , Delta = 1 , mu = 0.5, flux= 0 , phase = 0)

# In my units e = h = 1. So Phi_0 , the elementary flux quantum is 1 and the superconducting flux quantum is 1/2.
# Function for computing normal S-matrix. 
def compute_normal_s_matrix(params):
    B = params.flux/((params.W -1 )*(params.L - 1))
    peierlphase = B*params.a*np.pi
    syst = kwant.Builder()
    lat = kwant.lattice.square(a = params.a)
    for i in range(params.W):
        for j in range(params.L):
            syst[lat(i , j)] = 4*params.t - params.mu
            if i > 0:
                syst[lat(i , j) , lat(i-1 , j)] = -params.t
            if j > 0:
                syst[lat(i , j) , lat(i , j-1)] = -params.t*np.exp(1j*i*peierlphase)
    
    # Construct normal leads, with no magnetic field to account for screening.
    lead = kwant.Builder(kwant.TranslationalSymmetry((0 , -params.a)))
    for i in range(params.W):
        lead[lat(i , 0)] = 4*params.t - params.mu
        if i > 0:
            lead[lat(i , 0) , lat(i-1 , 0)] = -params.t
        lead[lat(i , 1) , lat(i , 0)] = -params.t
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    # Compute S-matrix of the normal region between leads. In the short junction regime , we can calcualte it at E = 0.
    s_matrix = kwant.smatrix(syst , 0)
    return s_matrix

In [29]:
s_matrix_test  = compute_normal_s_matrix(params)

In [ ]:
# Compute Andreev reflection matrix:

